#Aplicación de PCA

En este notebook veremos la aplicación de PCA a un set de datos

###Cargamos las librerias necesarias

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

###Cargamos el data set.

El set de datos USArrests contiene el porcentaje de asaltos (Assault), asesinatos (Murder) y secuestros (Rape) por cada 100,000 habitantes para cada uno de los 50 estados de USA (1973). Además, también incluye el porcentaje de la población de cada estado que vive en zonas rurales (UrbanPoP).

In [ ]:
USArrests = sm.datasets.get_rdataset("USArrests", "datasets")
datos = USArrests.data
datos.head()

###Vemos los tipos de datos.

Todo se ve en orden ya que son numéricos

In [ ]:
datos.info()

In [ ]:
datos.describe()

Vemos que la escala de los datos es muy diferente. Es necesario escalar los datos. Se deben dejar con media cero y desviación estándar 1.

Por defecto, PCA() centra los valores pero no los escala. Esto es importante ya que, si las variables tienen distinta dispersión, como en este caso, es necesario escalarlas.

###Se escalan los datos

In [ ]:
# Se crea primero el objeto Standard Scaler
scaler = StandardScaler()

# Luego "entrenamos" con los datos para obtener los parámetros del escalamiento
scaler.fit(datos)

# Finalmente aplicamos el escalamiento
array_scaler = scaler.transform(datos)

In [ ]:
#Corroboramos que dio resultado
pd.DataFrame(array_scaler).describe()

###Aplicamos PCA

[Acá](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) puede encontrar información de la función PCA

In [ ]:
# Al no incluir el número de componentes, lo que hace PCA es conservarlos todos
pca = PCA()

In [ ]:
pca.fit(array_scaler)

###Un poco de interpretación:

Una vez entrenado el objeto PCA, pude accederse a toda la información de las componentes creadas.

components_ contiene el valor que definen cada componente (eigenvector). Las filas se corresponden con las componentes principales (ordenadas de mayor a menor varianza explicada). Las filas se corresponden con las variables de entrada.

In [ ]:
# Se combierte el array a dataframe para añadir nombres a los ejes.
pd.DataFrame(
    data    = pca.components_,
    columns = datos.columns,
    index   = ['PC1', 'PC2', 'PC3', 'PC4']
)

Analizar con detalle el vector que forma cada componente puede ayudar a interpretar qué tipo de información recoge cada una de ellas. Por ejemplo, la primera componente es el resultado de la siguiente combinación lineal de las variables originales:

**PC1=0.535899 Murder+0.583184 Assault+0.278191 UrbanPop+0.543432 Rape**


Los pesos asignados en la primera componente a las variables Assault, Murder y Rape son aproximadamente iguales entre ellos y superiores al asignado a UrbanPoP. Esto significa que la primera componente recoge mayoritariamente la información correspondiente a los delitos. En la segunda componente, es la variable UrbanPoP es la que tiene el peso mayor (aunque sea negativo), por lo que se corresponde principalmente con el nivel de urbanización del estado. Si bien en este ejemplo la interpretación de las componentes es bastante clara, no en todos los casos ocurre lo mismo, sobre todo a medida que aumenta el número de variables.

Una vez calculadas las componentes principales, se puede conocer la varianza explicada por cada una de ellas, la proporción respecto al total y la proporción de varianza acumulada. Esta información está almacenada en los atributos **explained_variance_** y **explained_variance_ratio_** del modelo.

In [ ]:
print(pca.explained_variance_ratio_)

En orden, PC1 explica la varianza total en un 62%

Si sumamos todo, debe dar un 100% (o casi un 100%)

In [ ]:
print(sum(pca.explained_variance_ratio_)*100)

Podemos obtener la varianza acumulada simplemente sumando y acumulando lo aportado por cada componente.

In [ ]:
pca.explained_variance_ratio_.cumsum()

Esto nos dice por ejemplo, que solo necesitamos 2 dimensiones (en el nuevo espacio de coordenadas) para explicar el 86.8% de la varianza total.

Una vez hecho esto, podemos transformar los datos originales al espacio nuevo creado.

La transformación es el resultado de multiplicar los vectores que definen cada componente con el valor de las variables.

In [ ]:
# Aplicamos la transformación
proyecciones = pca.transform(datos)
# Lo transformamos a DataFrame, ya que queda como arreglo.
proyecciones = pd.DataFrame(proyecciones,columns = ['PC1', 'PC2', 'PC3', 'PC4'], index = datos.index)
proyecciones.head()

###Comparemos con el original

In [ ]:
datos.head()

Acá podemos decidir con cuantos componentes nos quedamos y así reducimos la dimensión. Eso dependiendo de con cuanta varianza explicada queremos quedarnos.

Finalmente, se puede reconstruir al espacio original una vez estando en el espacio transformado. Esto se hace con el método **inverse_transform()**. Es importante tener en cuenta que, la reconstrucción, solo será completa si se han incluido todas las componentes.

In [ ]:
recostruccion = pca.inverse_transform(proyecciones)
recostruccion = pd.DataFrame(recostruccion, columns = datos.columns, index = datos.index)
recostruccion.head()